<a href="https://colab.research.google.com/github/etltda/colabs/blob/main/Copy_of_Treinamento_Tacotron_PTBR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Colab de Treinamento Tacotron 2 (PT-BR)**

### Organizado e simplificado por mega b#6696 | [Tacotron 2](https://github.com/NVIDIA/tacotron2)

### Suporte para acentuação e tradução por Cris140
#### Agradecimentos ao IBob012 pelos avisos e ajuda no final.

### Código originalmente feito por Cookie para o Pony Preservation Project

---

## [Notebook para sintetizar](https://colab.research.google.com/drive/1-XWvLVhD11ZFosHsEqLnhiF58y-LlQWh) para testar seu modelo.

### **Aviso:**  Você pode encontrar dificuldades ao enviar arquivos (> 1 MB) se usar um navegador que não seja baseado em chromium.

# Celulas opcionais (reexibir clicando na seta à esquerda)

In [ ]:
#@markdown # Veja qual tipo de gpu
#@markdown ### Resete o ambiente de  execução caso ele nao der uma placa boa

#@markdown ---

#@markdown ## Não é recomendado usar a placa K80 (Demorada)

!nvidia-smi -L
#@markdown P100 = Adequada

#@markdown V100 = Adequada

#@markdown T4 = Adequada

#@markdown P4 = Adequada

#@markdown K80 = Não adequada

#@markdown ---

In [ ]:
#@markdown # Anti-Disconexão no Google Colab
#@markdown ## Execute-o para interromper a desconexão automática (no entanto, ele se desconectará após 6 a 12 horas).

import IPython
js_code = '''
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}
setInterval(ClickConnect,60000)
'''
display(IPython.display.Javascript(js_code))

In [ ]:
#@markdown # Descompacte o arquivo para descompactar wavs (Faça isso depois do segundo passo no treinamento)
#@markdown ### Se você tiver muitos arquivos wav, compacte-os todos em um arquivo localmente em seu sistema, faça o upload e copie o caminho.
#@markdown ---

import os

zip_file_path = "/content/wavs.zip" #@param {type:"string"}

if os.path.isdir("/content/tacotron2/wavs"):
  !unzip -j $zip_file_path -d /content/tacotron2/wavs
else:
  print("Failed: You must run the second step in the training first, before running this.")



# Treinamento

In [ ]:
#@markdown ## **1** Monte seu Google Drive

#Google Drive Authentication Token
from google.colab import drive
drive.mount('drive', force_remount=True)

In [ ]:
%%capture
#@markdown ## **2** Depois de executar esta célula, insira seus arquivos WAV (devem estar em 22.05Khz, Mono) na pasta tacotron-> wavs.

#@markdown ---

#@markdown ## Se você tiver muitos arquivos wav, simplesmente use a célula opcional: "Descompacte o arquivo para descompactar wavs" depois de executar essa célula. 

#@markdown #### Esta célula vai instalar tacotron 2 e suas dependencias.
%tensorflow_version 1.x
import os
!pip install phonemizer
!git clone -q https://github.com/etltda/tacotron2
!pip install git+https://github.com/etltda/num2words
os.chdir('tacotron2')
!git submodule init
!git submodule update
!pip install -q unidecode tensorboardX
!apt-get install pv
!apt-get install jq
!pip install --upgrade --no-cache-dir gdown
!wget https://raw.githubusercontent.com/tonikelope/megadown/master/megadown -O megadown.sh
!chmod 755 megadown.sh

%matplotlib inline
import os
if os.getcwd() != '/content/tacotron2':
    os.chdir('tacotron2')
import time
import argparse
import math
from numpy import finfo

import torch
from distributed import apply_gradient_allreduce
import torch.distributed as dist
from torch.utils.data.distributed import DistributedSampler
from torch.utils.data import DataLoader

from model import Tacotron2
from data_utils import TextMelLoader, TextMelCollate
from loss_function import Tacotron2Loss
from logger import Tacotron2Logger
from hparams import create_hparams
 
import random
import numpy as np

import layers
from utils import load_wav_to_torch, load_filepaths_and_text
from text import text_to_sequence
from math import e
#from tqdm import tqdm # Terminal
#from tqdm import tqdm_notebook as tqdm # Legacy Notebook TQDM
from tqdm.notebook import tqdm # Modern Notebook TQDM
from distutils.dir_util import copy_tree
import matplotlib.pylab as plt

def download_from_google_drive(file_id, file_name):
  # download a file from the Google Drive link
  !rm -f ./cookie
  !curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id={file_id}" > /dev/null
  confirm_text = !awk '/download/ {print $NF}' ./cookie
  confirm_text = confirm_text[0]
  !curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm={confirm_text}&id={file_id}" -o {file_name}

def create_mels():
    print("Generating Mels")
    stft = layers.TacotronSTFT(
                hparams.filter_length, hparams.hop_length, hparams.win_length,
                hparams.n_mel_channels, hparams.sampling_rate, hparams.mel_fmin,
                hparams.mel_fmax)
    def save_mel(filename):
        audio, sampling_rate = load_wav_to_torch(filename)
        if sampling_rate != stft.sampling_rate:
            raise ValueError("{} {} SR doesn't match target {} SR".format(filename, 
                sampling_rate, stft.sampling_rate))
        audio_norm = audio / hparams.max_wav_value
        audio_norm = audio_norm.unsqueeze(0)
        audio_norm = torch.autograd.Variable(audio_norm, requires_grad=False)
        melspec = stft.mel_spectrogram(audio_norm)
        melspec = torch.squeeze(melspec, 0).cpu().numpy()
        np.save(filename.replace('.wav', ''), melspec)

    import glob
    wavs = glob.glob('wavs/*.wav')
    for i in tqdm(wavs):
        save_mel(i)


def reduce_tensor(tensor, n_gpus):
    rt = tensor.clone()
    dist.all_reduce(rt, op=dist.reduce_op.SUM)
    rt /= n_gpus
    return rt


def init_distributed(hparams, n_gpus, rank, group_name):
    assert torch.cuda.is_available(), "Distributed mode requires CUDA."
    print("Initializing Distributed")

    # Set cuda device so everything is done on the right GPU.
    torch.cuda.set_device(rank % torch.cuda.device_count())

    # Initialize distributed communication
    dist.init_process_group(
        backend=hparams.dist_backend, init_method=hparams.dist_url,
        world_size=n_gpus, rank=rank, group_name=group_name)

    print("Done initializing distributed")


def prepare_dataloaders(hparams):
    # Get data, data loaders and collate function ready
    trainset = TextMelLoader(hparams.training_files, hparams)
    valset = TextMelLoader(hparams.validation_files, hparams)
    collate_fn = TextMelCollate(hparams.n_frames_per_step)

    if hparams.distributed_run:
        train_sampler = DistributedSampler(trainset)
        shuffle = False
    else:
        train_sampler = None
        shuffle = True

    train_loader = DataLoader(trainset, num_workers=1, shuffle=shuffle,
                              sampler=train_sampler,
                              batch_size=hparams.batch_size, pin_memory=False,
                              drop_last=True, collate_fn=collate_fn)
    return train_loader, valset, collate_fn


def prepare_directories_and_logger(output_directory, log_directory, rank):
    if rank == 0:
        if not os.path.isdir(output_directory):
            os.makedirs(output_directory)
            os.chmod(output_directory, 0o775)
        logger = Tacotron2Logger(os.path.join(output_directory, log_directory))
    else:
        logger = None
    return logger


def load_model(hparams):
    model = Tacotron2(hparams).cuda()
    if hparams.fp16_run:
        model.decoder.attention_layer.score_mask_value = finfo('float16').min

    if hparams.distributed_run:
        model = apply_gradient_allreduce(model)

    return model


def warm_start_model(checkpoint_path, model, ignore_layers):
    assert os.path.isfile(checkpoint_path)
    print("Warm starting model from checkpoint '{}'".format(checkpoint_path))
    checkpoint_dict = torch.load(checkpoint_path, map_location='cpu')
    model_dict = checkpoint_dict['state_dict']
    if len(ignore_layers) > 0:
        model_dict = {k: v for k, v in model_dict.items()
                      if k not in ignore_layers}
        dummy_dict = model.state_dict()
        dummy_dict.update(model_dict)
        model_dict = dummy_dict
    model.load_state_dict(model_dict)
    return model


def load_checkpoint(checkpoint_path, model, optimizer):
    assert os.path.isfile(checkpoint_path)
    print("Loading checkpoint '{}'".format(checkpoint_path))
    checkpoint_dict = torch.load(checkpoint_path, map_location='cpu')
    model.load_state_dict(checkpoint_dict['state_dict'])
    optimizer.load_state_dict(checkpoint_dict['optimizer'])
    learning_rate = checkpoint_dict['learning_rate']
    iteration = checkpoint_dict['iteration']
    print("Loaded checkpoint '{}' from iteration {}" .format(
        checkpoint_path, iteration))
    return model, optimizer, learning_rate, iteration


def save_checkpoint(model, optimizer, learning_rate, iteration, filepath):
    import random
    if random.random() > 0.85:
        print("Saving model and optimizer state at iteration {} to {}".format(
            iteration, filepath))
        try:
            torch.save({'iteration': iteration,
                    'state_dict': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'learning_rate': learning_rate}, filepath)
        except KeyboardInterrupt:
            print("interrupt received while saving, waiting for save to complete.")
            torch.save({'iteration': iteration,'state_dict': model.state_dict(),'optimizer': optimizer.state_dict(),'learning_rate': learning_rate}, filepath)
        print("Model Saved")

def plot_alignment(alignment, info=None):
    %matplotlib inline
    fig, ax = plt.subplots(figsize=(int(alignment_graph_width/100), int(alignment_graph_height/100)))
    im = ax.imshow(alignment, cmap='inferno', aspect='auto', origin='lower',
                   interpolation='none')
    ax.autoscale(enable=True, axis="y", tight=True)
    fig.colorbar(im, ax=ax)
    xlabel = 'Decoder timestep'
    if info is not None:
        xlabel += '\n\n' + info
    plt.xlabel(xlabel)
    plt.ylabel('Encoder timestep')
    plt.tight_layout()
    fig.canvas.draw()
    plt.show()

def validate(model, criterion, valset, iteration, batch_size, n_gpus,
             collate_fn, logger, distributed_run, rank, epoch, start_eposh, learning_rate):
    """Handles all the validation scoring and printing"""
    model.eval()
    with torch.no_grad():
        val_sampler = DistributedSampler(valset) if distributed_run else None
        val_loader = DataLoader(valset, sampler=val_sampler, num_workers=1,
                                shuffle=False, batch_size=batch_size,
                                pin_memory=False, collate_fn=collate_fn)

        val_loss = 0.0
        for i, batch in enumerate(val_loader):
            x, y = model.parse_batch(batch)
            y_pred = model(x)
            loss = criterion(y_pred, y)
            if distributed_run:
                reduced_val_loss = reduce_tensor(loss.data, n_gpus).item()
            else:
                reduced_val_loss = loss.item()
            val_loss += reduced_val_loss
        val_loss = val_loss / (i + 1)

    model.train()
    if rank == 0:
        print("Epoch: {} Validation loss {}: {:9f}  Time: {:.1f}m LR: {:.6f}".format(epoch, iteration, val_loss,(time.perf_counter()-start_eposh)/60, learning_rate))
        logger.log_validation(val_loss, model, y, y_pred, iteration)
        if hparams.show_alignments:
            %matplotlib inline
            _, mel_outputs, gate_outputs, alignments = y_pred
            idx = random.randint(0, alignments.size(0) - 1)
            plot_alignment(alignments[idx].data.cpu().numpy().T)

def train(output_directory, log_directory, checkpoint_path, warm_start, n_gpus,
          rank, group_name, hparams, log_directory2):
    """Training and validation logging results to tensorboard and stdout

    Params
    ------
    output_directory (string): directory to save checkpoints
    log_directory (string) directory to save tensorboard logs
    checkpoint_path(string): checkpoint path
    n_gpus (int): number of gpus
    rank (int): rank of current gpu
    hparams (object): comma separated list of "name=value" pairs.
    """
    if hparams.distributed_run:
        init_distributed(hparams, n_gpus, rank, group_name)

    torch.manual_seed(hparams.seed)
    torch.cuda.manual_seed(hparams.seed)

    model = load_model(hparams)
    learning_rate = hparams.learning_rate
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,
                                 weight_decay=hparams.weight_decay)

    if hparams.fp16_run:
        from apex import amp
        model, optimizer = amp.initialize(
            model, optimizer, opt_level='O2')

    if hparams.distributed_run:
        model = apply_gradient_allreduce(model)

    criterion = Tacotron2Loss()

    logger = prepare_directories_and_logger(
        output_directory, log_directory, rank)

    train_loader, valset, collate_fn = prepare_dataloaders(hparams)

    # Load checkpoint if one exists
    iteration = 0
    epoch_offset = 0
    if checkpoint_path is not None and os.path.isfile(checkpoint_path):
        if warm_start:
            model = warm_start_model(
                checkpoint_path, model, hparams.ignore_layers)
        else:
            model, optimizer, _learning_rate, iteration = load_checkpoint(
                checkpoint_path, model, optimizer)
            if hparams.use_saved_learning_rate:
                learning_rate = _learning_rate
            iteration += 1  # next iteration is iteration + 1
            epoch_offset = max(0, int(iteration / len(train_loader)))
    else:
      os.path.isfile("/content/tacotron2/pretrained_model")
      %cd /dev/null
      !/content/tacotron2/megadown.sh https://mega.nz/#!WXY3RILA!KyoGHtfB_sdhmLFoykG2lKWhh0GFdwMkk7OwAjpQHRo --o pretrained_model
      %cd /content/tacotron2
      model = warm_start_model("/content/tacotron2/pretrained_model", model, hparams.ignore_layers)
      # download LJSpeech pretrained model if no checkpoint already exists
    
    start_eposh = time.perf_counter()
    learning_rate = 0.0
    model.train()
    is_overflow = False
    # ================ MAIN TRAINNIG LOOP! ===================
    for epoch in tqdm(range(epoch_offset, hparams.epochs)):
        print("\nStarting Epoch: {} Iteration: {}".format(epoch, iteration))
        start_eposh = time.perf_counter() # eposh is russian, not a typo
        for i, batch in tqdm(enumerate(train_loader), total=len(train_loader)):
            start = time.perf_counter()
            if iteration < hparams.decay_start: learning_rate = hparams.A_
            else: iteration_adjusted = iteration - hparams.decay_start; learning_rate = (hparams.A_*(e**(-iteration_adjusted/hparams.B_))) + hparams.C_
            learning_rate = max(hparams.min_learning_rate, learning_rate) # output the largest number
            for param_group in optimizer.param_groups:
                param_group['lr'] = learning_rate

            model.zero_grad()
            x, y = model.parse_batch(batch)
            y_pred = model(x)

            loss = criterion(y_pred, y)
            if hparams.distributed_run:
                reduced_loss = reduce_tensor(loss.data, n_gpus).item()
            else:
                reduced_loss = loss.item()
            if hparams.fp16_run:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()

            if hparams.fp16_run:
                grad_norm = torch.nn.utils.clip_grad_norm_(
                    amp.master_params(optimizer), hparams.grad_clip_thresh)
                is_overflow = math.isnan(grad_norm)
            else:
                grad_norm = torch.nn.utils.clip_grad_norm_(
                    model.parameters(), hparams.grad_clip_thresh)

            optimizer.step()

            if not is_overflow and rank == 0:
                duration = time.perf_counter() - start
                logger.log_training(
                    reduced_loss, grad_norm, learning_rate, duration, iteration)
                #print("Batch {} loss {:.6f} Grad Norm {:.6f} Time {:.6f}".format(iteration, reduced_loss, grad_norm, duration), end='\r', flush=True)

            iteration += 1
        validate(model, criterion, valset, iteration,
                 hparams.batch_size, n_gpus, collate_fn, logger,
                 hparams.distributed_run, rank, epoch, start_eposh, learning_rate)
        save_checkpoint(model, optimizer, learning_rate, iteration, checkpoint_path)
        if log_directory2 != None:
            copy_tree(log_directory, log_directory2)
def check_dataset(hparams):
    from utils import load_wav_to_torch, load_filepaths_and_text
    import os
    import numpy as np
    def check_arr(filelist_arr):
        for i, file in enumerate(filelist_arr):
            if len(file) > 2:
                print("|".join(file), "\nhas multiple '|', this may not be an error.")
            if hparams.load_mel_from_disk and '.wav' in file[0]:
                print("[WARNING]", file[0], " in filelist while expecting .npy .")
            else:
                if not hparams.load_mel_from_disk and '.npy' in file[0]:
                    print("[WARNING]", file[0], " in filelist while expecting .wav .")
            if (not os.path.exists(file[0])):
                print("|".join(file), "\n[WARNING] does not exist.")
            if len(file[1]) < 3:
                print("|".join(file), "\n[info] has no/very little text.")
            if not ((file[1].strip())[-1] in r"!?,.;:"):
                print("|".join(file), "\n[info] has no ending punctuation.")
            mel_length = 1
            if hparams.load_mel_from_disk and '.npy' in file[0]:
                melspec = torch.from_numpy(np.load(file[0], allow_pickle=True))
                mel_length = melspec.shape[1]
            if mel_length == 0:
                print("|".join(file), "\n[WARNING] has 0 duration.")
    print("Checking Training Files")
    audiopaths_and_text = load_filepaths_and_text(hparams.training_files) # get split lines from training_files text file.
    check_arr(audiopaths_and_text)
    print("Checking Validation Files")
    audiopaths_and_text = load_filepaths_and_text(hparams.validation_files) # get split lines from validation_files text file.
    check_arr(audiopaths_and_text)
    print("Finished Checking")
#@markdown ####*Recomendado para início de treinamento, desativar se estiver restaurando um checkpoint.*
warm_start = "Sim" #@param ["Sim", "Nao"]
if warm_start=="Sim":
    warm_start=True
else:
    warm_start=False
n_gpus=1
rank=0
group_name=None

# ---- DEFAULT PARAMETERS DEFINED HERE ----
hparams = create_hparams()
model_filename = 'current_model'
hparams.training_files = "filelists/clipper_train_filelist.txt"
hparams.validation_files = "filelists/clipper_val_filelist.txt"
#hparams.use_mmi=True,          # not used in this notebook
#hparams.use_gaf=True,          # not used in this notebook
#hparams.max_gaf=0.5,           # not used in this notebook
#hparams.drop_frame_rate = 0.2  # not used in this notebook
hparams.p_attention_dropout=0.1
hparams.p_decoder_dropout=0.1
hparams.decay_start = 15000
hparams.A_ = 5e-4
hparams.B_ = 8000
hparams.C_ = 0
hparams.min_learning_rate = 1e-5
generate_mels = True
hparams.show_alignments = True
alignment_graph_height = 600
alignment_graph_width = 1000
hparams.batch_size = 32
hparams.load_mel_from_disk = True
hparams.epochs = 10000
torch.backends.cudnn.enabled = hparams.cudnn_enabled
torch.backends.cudnn.benchmark = hparams.cudnn_benchmark
output_directory = '/content/drive/My Drive/colab/outdir' # Location to save Checkpoints
log_directory = '/content/tacotron2/logs' # Location to save Log files locally
log_directory2 = '/content/drive/My Drive/colab/logs' # Location to copy log files (done at the end of each epoch to cut down on I/O)
checkpoint_path = output_directory+(r'/')+model_filename

# ---- Replace .wav with .npy in filelists ----
!sed -i -- 's,.wav|,.npy|,g' filelists/*.txt
!sed -i -- 's,.wav|,.npy|,g' {hparams.training_files}
!sed -i -- 's,.wav|,.npy|,g' {hparams.validation_files}
# ---- Replace .wav with .npy in filelists ----

%cd /content/tacotron2

data_path = 'wavs'
!mkdir {data_path}


In [ ]:
import os
import os.path
from pathlib import Path
from itertools import chain
from glob import glob
#@markdown ## **3** Configurar os parâmetros do modelo
#@markdown ---
#@markdown ####  Nome desejado para o modelo
model_filename = "" #@param {type: "string"}
#@markdown #### Faça upload da sua transcrição/text para a pasta tacotron/filelists e dê o clique direito -> copiar caminho
Training_file = "filelists/list.txt" #@param {type: "string"}
hparams.training_files = Training_file
hparams.validation_files = Training_file
# hparams to Tune
#hparams.use_mmi=True,          # not used in this notebook
#hparams.use_gaf=True,          # not used in this notebook
#hparams.max_gaf=0.5,           # not used in this notebook
#hparams.drop_frame_rate = 0.2  # not used in this notebook
hparams.p_attention_dropout=0.1
hparams.p_decoder_dropout=0.1

# Learning Rate             # https://www.desmos.com/calculator/ptgcz4vzsw / http://boards.4channel.org/mlp/thread/34778298#p34789030
hparams.decay_start = 15000         # wait till decay_start to start decaying learning rate
hparams.A_ = 5e-4                   # Start/Max Learning Rate
hparams.B_ = 8000                   # Decay Rate
hparams.C_ = 0                      # Shift learning rate equation by this value
hparams.min_learning_rate = 1e-5    # Min Learning Rate

# Quality of Life
generate_mels = True
hparams.show_alignments = True
alignment_graph_height = 600
alignment_graph_width = 1000

#@markdown #### O tamanho do batch, abaixe se não estiver com muita ram.
#@markdown ###### Se você tiver uma Tesla T4, é recomendado usar 14 como o tamanho do batch.
hparams.batch_size = 18 #@param {type: "integer"}
hparams.load_mel_from_disk = True
 # Layers to reset (None by default, other than foreign languages this param can be ignored)

#@markdown #### O total de epochs para treinar. Não é recomendado mudar.

##@markdown #### Tanto de epochs antes de parar, preferivel um grande valor para parar.
hparams.epochs = 250 #@param {type: "integer"}

torch.backends.cudnn.enabled = hparams.cudnn_enabled
torch.backends.cudnn.benchmark = hparams.cudnn_benchmark

#@markdown #### Onde salvar seu modelo quando treinar.
output_directory = '/content/drive/My Drive/colab/outdir' #@param {type: "string"}
log_directory = '/content/tacotron2/logs' # Location to save Log files locally
log_directory2 = '/content/drive/My Drive/colab/logs' # Location to copy log files (done at the end of each epoch to cut down on I/O)
checkpoint_path = output_directory+(r'/')+model_filename


#@markdown ####  Se quiser, pode utilizar um modelo base para um alinhamento mais rápido
#@markdown ###### Se não quiser, só desmarçar a opção, mas é extremamente recomendado utilizar
newpath = r'/content/drive/MyDrive/colab1/outdir/' 
if not os.path.exists(newpath):
    os.makedirs(newpath)
modelo_base = True #@param{type:"boolean"}
status_arquivo = os.path.isfile('/content/drive/MyDrive/colab/outdir/'+model_filename)
if modelo_base == True:
    if status_arquivo == True:
      print("Já tem um arquivo com esse nome no seu drive. Se quiser utilizar o modelo base terá de apagar/renomear esse arquivo")  
    else:
      os.system("gdown --id 1-02A_qDM9Q7XI74jVO5UYKf4rKXD30V8 -O /content/drive/MyDrive/colab/outdir/"+model_filename)
#@markdown ---

In [ ]:
#@title ## **3.1** Rode para checar se algum dos arquivos não está no formato correto
#@markdown Obs: se já estiverem corretos, não roda, pode ocorrer problemas.
%%capture
import os 
import wave

can_scan = False
folder_name = 'wavs'
folder_path = f'{folder_name}/';

if os.path.isdir(folder_path):
  print(f'Folder name set to {folder_name}.')
  can_scan = True
else:
  print('Invalid folder path.')

if can_scan == True:
  resample_list = []
  for file_name in os.listdir(folder_path):
    with wave.open(folder_path + file_name, "rb") as wave_file:
        frame_rate = wave_file.getframerate()
        if frame_rate == 22050:
          print(f'{file_name} does not require resampling.')
        else:
          print(f'{file_name} requires resampling.')
          resample_list.append(file_name)
else:
  print('Please return to step 2 before continuing step 3.')

if len(resample_list) > 0:
  need_to_resample = True
  print(f'\nScan completed. Please continue to Step 4 to resample {len(resample_list)} file(s).')
else:
  print('\nThere are no files to resample.')
  need_to_resample = False
if need_to_resample == False:
  print('There is nothing to resample.')
  exit()

new_rate = 22050

for item in resample_list:
  os.system(f'ffmpeg -y -i {folder_path}{item} -ar {new_rate} -ac 1 {folder_path}{item} -preset ultrafast -threads 0') # Converts to Mono also
  print(f'Resampled file {item}.')

In [ ]:
#@markdown ## **4** Converter arquivos WAV em espectrogramas mel (Rode só uma vez)
#@markdown ### *Também procura por arquivos faltando*
import re
import num2words
import sys
from contextlib import redirect_stdout
import fileinput

f_input = open('/content/tacotron2/filelists/list.txt','r')
text = f_input.readlines()
#print(text)
text2 = len(text)
textmacaca = ""
#print(text2)
for number in range(0,text2):
 macaca = number
 if(str(macaca) + ".wav") in (text[number]):
   pass
 else:
  textmacaca = str(text[number])
  textmacaca2 = textmacaca.split("|")
  #print(textmacaca2)
  textmacaca2[1] = re.sub(r"(\d+)", lambda x: num2words.num2words(int(x.group(0)), lang='pt_BR'), textmacaca2[1],)
  #print(textmacaca2[1])
  text[number]  = (str(textmacaca2[0] + "|" + textmacaca2[1].lower()))
  #print(text[number])

  #print(text[number])
textfinal = " "
for number in range(text2):
 textfinal += text[number]
textfinal = textfinal.replace('[','')
textfinal = textfinal.replace("',",'')
textfinal = textfinal.replace("'",'')
textfinal = textfinal.replace(']','')
textfinal = textfinal.replace('\\n','\n')
textfinal = textfinal.replace(' wavs','wavs')
t = textfinal
"".join([s for s in t.strip().splitlines(True) if s.strip()])
print(textfinal, file=open("/content/tacotron2/filelists/list1.txt", "a"))


!rm /content/tacotron2/filelists/list.txt
!mv /content/tacotron2/filelists/list1.txt /content/tacotron2/filelists/list.txt
print("Gerando mels")
if generate_mels:
    create_mels()

print("Checando por arquivos faltando")
# ---- Replace .wav with .npy in filelists ----
!sed -i -- 's,.wav|,.npy|,g' {hparams.training_files}; sed -i -- 's,.wav|,.npy|,g' {hparams.validation_files}

check_dataset(hparams)

In [ ]:
#@markdown # **5** Começar treinamento

print('FP16 Run:', hparams.fp16_run)
print('Dynamic Loss Scaling:', hparams.dynamic_loss_scaling)
print('Distributed Run:', hparams.distributed_run)
print('cuDNN Enabled:', hparams.cudnn_enabled)
print('cuDNN Benchmark:', hparams.cudnn_benchmark)
train(output_directory, log_directory, checkpoint_path,
      warm_start, n_gpus, rank, group_name, hparams, log_directory2)

# **Treinamento bom se parece com isso aqui:**

![img.png](https://media.discordapp.net/attachments/835971020569051216/851469553355587614/download_2.png)

JS to prevent idle timeout:

Press F12 OR CTRL + SHIFT + I OR right click on this website -> inspect.
Then click on the console tab and paste in the following code.

```javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}
setInterval(ClickConnect,60000)
```

Aproveitando isso, aqui tá alguns erros que você pode pegar e como consertá-los

![img.png](https://media.discordapp.net/attachments/841732367962210344/889163051314602014/Screenshot_107.png?width=924&height=218)

CUDA out of memory é quando o seu batch size é alto demais pra o GPU dar conta, vai ter que abaixá-lo ou verificar se nos seus arquivos tem algum audio muito longo(acima de 12 segundos).
As vezes mesmo abaixando até literalmente 1 continua dando erro, eu recomendo quando pegar esse erro abaixar alguns digitos e redefinir o ambiente que é mais fácil.

![img.png](https://media.discordapp.net/attachments/841732367962210344/889158394441769010/Screenshot_106.png?width=924&height=348)

list index out of range é quando na sua transcrição tem uma linha vazia. Geralmente essas linhas ficam no fim da transcript acidentalmente, remova e dê upload delas novamente (QUANDO FIZER ISSO RODE A ETAPA 4 DENOVO)

Queria adicionar o erro Cannot load file containing pickled data, isso também significa que tem uma linha vazia na transcrição, diferença é que você ganha esse erro em particular tentando treinar.

![img.png](https://media.discordapp.net/attachments/841732367962210344/889042673602813952/unknown.png?width=652&height=469)

shape is invalid for input size é quando um ou mais áudios que você colocou não são stereo. Ache algum programa de converter áudios para as configurações corretas (22hz PCM Uncompressed Mono) e pra evitar dor de cabeça pra substituir que áudios estavam incorretos é melhor só reiniciar o ambiente mesmo.

![img.png](https://media.discordapp.net/attachments/841732367962210344/888658737332637706/unknown.png?width=919&height=432)

O erro que me aterrorizou quando eu queria treinar o Chills. No such file or directory: '\uffefwavs/1.npy' é causado quando a sua lista está em UTF-8, quando ela devia estar em UTF-8 without BOM. Pra solucionar isso é usando algum editor de texto e mudando pra UTF-8 without BOM, eu uso notepad++ pra isso pessoalmente, não tenha medo de pesquisar no Google como fazer.


Isso é basicamente todos os erros super conhecidos, se você não entendeu como se soluciona alguma das coisas aqui pergunta pra gente, a gente não morde.
